In [256]:
pip install textfsm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     -------------------------------------- 44.7/44.7 kB 200.1 kB/s eta 0:00:00
     ------------------------------------ 840.9/840.9 kB 209.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492055 sha256=463ec9aeaa9b4fead206f53f0423663c5db85d87acb7574899f5573fa64bdbd0
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\6e\54\16\2ab500319d13efeeee4b2007aab5a01589dc775fb1a1f617c5
Successfully built future
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
 
import telnetlib
from textfsm import TextFSM
import re,json


In [2]:
def readCfg(filename):
    """读取配置文件"""
    # 读取json文件内容,返回字典格式
    json_data = {}
    with open(filename,'r',encoding='utf8')as fp:
        json_data = json.load(fp)
    return json_data
cfg = readCfg('sw_config.json')
cfg

{'交换机列表': [{'type': 'hw.s3700',
   'name': '3700',
   'ip': '192.168.11.1',
   'user': 'admin',
   'passwd': 'wsf2010'},
  {'type': 'h3c.s5560s',
   'name': '5560',
   'ip': '192.168.11.205',
   'user': 'admin',
   'passwd': 'wsf119900'},
  {'type': 'h3c.s1850',
   'name': 'lou7',
   'ip': '192.168.11.202',
   'user': 'admin',
   'passwd': 'wsf119900'},
  {'type': 'h3c.s1850',
   'name': 'lou3',
   'ip': '192.168.11.203',
   'user': 'admin',
   'passwd': 'wsf119900'},
  {'type': 'rg.nbs5552xg',
   'name': 'rg552',
   'ip': '192.168.11.200',
   'user': 'wsf2010',
   'passwd': 'wsf2010'}],
 '输出路径': './'}

In [3]:


one = cfg['交换机列表'][2]
one

{'type': 'h3c.s1850',
 'name': 'lou7',
 'ip': '192.168.11.202',
 'user': 'admin',
 'passwd': 'wsf119900'}

In [4]:
host=one['ip']
 
password1=one['passwd']

### 任意读取

In [31]:


out = tn.read_very_eager().decode('ascii')#读取缓冲区内的数据不等待  //  \r\r\n---- More ---- 
tn.write(' '.encode('ascii')+b"\n" )
out

' \r\n<lou7>'

In [32]:
print(out)

 
<lou7>


In [34]:
def save2file(s,fileNmae):
    """ 把str 存入文件 """
    with open(fileNmae, "w", encoding='utf-8') as f:
        f.write(str(s))
        f.close()

In [35]:
save2file(out,'a.txt')

In [ ]:
tn.write(one['user'].encode('ascii') )

In [12]:
tn.read_until(b"Password:",timeout=5)#//读取到




b'wsf119900'

In [77]:
tn.write(' '.encode('ascii')+b"\n" )

In [23]:
tn.write(b"display current-configuration\n")

## 预期读

In [75]:

outs = tn.expect([b"---- More ----"],timeout=2) 
outs

(0,
 <re.Match object; span=(1881, 1895), match=b'---- More ----'>,
 b'\r\r               \r192.168.34.79   f4b5-2024-f44c 34         GE1/0/11                 5     D    \r\r\n192.168.34.81   f4b5-2024-ebaf 34         GE1/0/11                 20    D    \r\r\n192.168.34.82   00e0-0b48-a717 34         GE1/0/11                 5     D    \r\r\n192.168.34.90   00e0-7070-de71 34         GE1/0/11                 19    D    \r\r\n192.168.34.102  e0d5-5efc-0d5f 34         GE1/0/11                 12    D    \r\r\n192.168.34.111  f4b5-2024-f365 34         GE1/0/11                 9     D    \r\r\n192.168.34.117  0c9d-920f-c668 34         GE1/0/11                 7     D    \r\r\n192.168.34.121  408d-5cfd-e739 34         GE1/0/11                 18    D    \r\r\n192.168.34.144  b42e-9944-2e6e 34         GE1/0/11                 5     D    \r\r\n192.168.34.147  a85e-45a0-0b0e 34         GE1/0/11                 5     D    \r\r\n192.168.34.150  6045-cb7f-c902 34         GE1/0/11                 5

In [74]:
#print(out)
print(outs[0],outs[2].decode('utf-8'))

-1 


In [26]:
tn.write('display mac-addr '.encode('ascii')+b"\n")

In [72]:
outs = tn.expect([b"\\x1b\[0m All\s+",b"---- More ----"],timeout=5) 
outs

(-1, None, b'')

In [5]:
 

def h3clogin(user,passwd,ip):
    """登入交换机 返回telnet"""
    outLs = []
    tn=telnetlib.Telnet(ip)
    out = tn.read_until(b"Username:",timeout=5)
    outLs.append(out)
    #--输入用户名
    tn.write(user.encode('ascii')+b"\n")
    out = tn.read_until(b"Password:",timeout=5)
    outLs.append(out)
    #--输入密码
    tn.write(passwd.encode('ascii')+b"\n")
    outs = tn.expect([b"<.*?>"],timeout=5)##等待直到读到预期的值
    outLs.append(outs[2])
    ## ------
    #print(outLs)
    return tn


    

In [6]:
tn = h3clogin(one['user'],one['passwd'],one['ip'])

In [80]:

def mac2tab(tn):
    """ h3c返回交换 arp表"""
    tabLs = []
    tn.write(b" display mac-address\n")
    while 1 :
        outLs = tn.expect([b"\r\r\n---- More ----"],timeout=5) #//[b"\r\r\n---- More ----"]
        if outLs[0]  >-1 :
            #还有下一个页
            tn.write(b" ")
            tabLs.append(outLs[2].decode('ascii'))
            #print(outLs)
        if outLs[0] < 0:
            # 完成读取
            #print(outLs)
            tabLs.append(outLs[2].decode('ascii'))
            break
    return tabLs


tabLs = arp2tab(tn)
                       
tabLs

['display mac-addr \r\nMAC ADDR        VLAN ID  STATE           PORT INDEX                AGING TIME(s)\r\na85e-455a-5a7c  1        Learned         GigabitEthernet1/0/24     AGING\r\n4cbd-8f79-1b53  1        Learned         GigabitEthernet1/0/23     AGING\r\ne0d5-5efc-0288  1        Learned         GigabitEthernet1/0/24     AGING\r\n00e0-4c52-032d  1        Learned         GigabitEthernet1/0/24     AGING\r\n40b0-765f-6e47  1        Learned         GigabitEthernet1/0/24     AGING\r\n04f9-f816-6b61  1        Learned         GigabitEthernet1/0/23     AGING\r\n7486-e218-1afe  1        Learned         GigabitEthernet1/0/24     AGING\r\n4cbd-8f79-1b76  1        Learned         GigabitEthernet1/0/23     AGING\r\nfcaa-1470-f02e  1        Learned         GigabitEthernet1/0/24     AGING\r\n4cbd-8f79-1b7a  1        Learned         GigabitEthernet1/0/23     AGING\r\nb42e-999d-1f99  1        Learned         GigabitEthernet1/0/24     AGING\r\ne062-90c9-1712  1        Learned         GigabitEthernet1

In [74]:
tabLs[2]

'\x1b[16D                \x1b[16Df000-006c-8155  1        Learned         GigabitEthernet1/0/15     AGING\r\n807c-62bb-4e1e  1        Learned         GigabitEthernet1/0/24     AGING\r\n5254-00d4-636c  1        Learned         GigabitEthernet1/0/24     AGING\r\nca8d-10bb-62ab  1        Learned         GigabitEthernet1/0/24     AGING\r\n2428-fddc-be9d  1        Learned         GigabitEthernet1/0/24     AGING\r\nb42e-99e6-3c49  1        Learned         GigabitEthernet1/0/24     AGING\r\nf000-006c-8167  1        Learned         GigabitEthernet1/0/5      AGING\r\n309c-2365-209e  1        Learned         GigabitEthernet1/0/24     AGING\r\n80fb-06c9-6db9  1        Learned         GigabitEthernet1/0/24     AGING\r\n0c9d-92cc-8c94  1        Learned         GigabitEthernet1/0/24     AGING\r\ne0d5-5e51-ea27  1        Learned         GigabitEthernet1/0/23     AGING\r\nb42e-99ea-043c  1        Learned         GigabitEthernet1/0/24     AGING\r\ne0d5-5e56-7edb  1        Learned         GigabitEtherne

In [81]:
len(tabLs)

10

In [52]:
tabStr = ''.join(tabLs)
tabStr

'display mac-addr \r\nMAC ADDR        VLAN ID  STATE           PORT INDEX                AGING TIME(s)\r\n4cbd-8f79-1b53  1        Learned         GigabitEthernet1/0/23     AGING\r\ne0d5-5efc-0288  1        Learned         GigabitEthernet1/0/24     AGING\r\n00e0-4c52-032d  1        Learned         GigabitEthernet1/0/24     AGING\r\n40b0-765f-6e47  1        Learned         GigabitEthernet1/0/24     AGING\r\n04f9-f816-6b61  1        Learned         GigabitEthernet1/0/23     AGING\r\n4cbd-8f79-1b76  1        Learned         GigabitEthernet1/0/23     AGING\r\nfcaa-1470-f02e  1        Learned         GigabitEthernet1/0/24     AGING\r\n4cbd-8f79-1b7a  1        Learned         GigabitEthernet1/0/23     AGING\r\nb42e-999d-1f99  1        Learned         GigabitEthernet1/0/24     AGING\r\ne062-90c9-1712  1        Learned         GigabitEthernet1/0/23     AGING\r\nf079-596c-908a  1        Learned         GigabitEthernet1/0/24     AGING\r\n7486-e218-0a99  1        Learned         GigabitEthernet1/

In [53]:
print(tabStr)

display mac-addr 
MAC ADDR        VLAN ID  STATE           PORT INDEX                AGING TIME(s)
4cbd-8f79-1b53  1        Learned         GigabitEthernet1/0/23     AGING
e0d5-5efc-0288  1        Learned         GigabitEthernet1/0/24     AGING
00e0-4c52-032d  1        Learned         GigabitEthernet1/0/24     AGING
40b0-765f-6e47  1        Learned         GigabitEthernet1/0/24     AGING
04f9-f816-6b61  1        Learned         GigabitEthernet1/0/23     AGING
4cbd-8f79-1b76  1        Learned         GigabitEthernet1/0/23     AGING
fcaa-1470-f02e  1        Learned         GigabitEthernet1/0/24     AGING
4cbd-8f79-1b7a  1        Learned         GigabitEthernet1/0/23     AGING
b42e-999d-1f99  1        Learned         GigabitEthernet1/0/24     AGING
e062-90c9-1712  1        Learned         GigabitEthernet1/0/23     AGING
f079-596c-908a  1        Learned         GigabitEthernet1/0/24     AGING
7486-e218-0a99  1        Learned         GigabitEthernet1/0/24     AGING
0017-6112-279e  1        

In [82]:
def list2str(tabLs): 
    """ h3c 多行合并成字符串"""
    tabStr = ''.join(tabLs)
    tabStr =  re.sub("  ---- More ----\\x1b\[16D\s+\\x1b\[16D", "", tabStr)
    tabStr =  re.sub("\r\r\n","\r\n", tabStr)   
    return tabStr


In [83]:
tabStr = list2str(tabLs)   

In [84]:
print(tabStr)

display mac-addr 
MAC ADDR        VLAN ID  STATE           PORT INDEX                AGING TIME(s)
a85e-455a-5a7c  1        Learned         GigabitEthernet1/0/24     AGING
4cbd-8f79-1b53  1        Learned         GigabitEthernet1/0/23     AGING
e0d5-5efc-0288  1        Learned         GigabitEthernet1/0/24     AGING
00e0-4c52-032d  1        Learned         GigabitEthernet1/0/24     AGING
40b0-765f-6e47  1        Learned         GigabitEthernet1/0/24     AGING
04f9-f816-6b61  1        Learned         GigabitEthernet1/0/23     AGING
7486-e218-1afe  1        Learned         GigabitEthernet1/0/24     AGING
4cbd-8f79-1b76  1        Learned         GigabitEthernet1/0/23     AGING
fcaa-1470-f02e  1        Learned         GigabitEthernet1/0/24     AGING
4cbd-8f79-1b7a  1        Learned         GigabitEthernet1/0/23     AGING
b42e-999d-1f99  1        Learned         GigabitEthernet1/0/24     AGING
e062-90c9-1712  1        Learned         GigabitEthernet1/0/23     AGING
f079-596c-908a  1        

In [86]:
def arp2rows(arpTab,tpl="display-mac-s1580.template"):
    """  解析表格字符串  返回json"""
    f = open(tpl)
    template = TextFSM(f)
    outTab = template.ParseText(arpTab)
    return outTab
arpLs = arp2rows(tabStr)

In [87]:
len(arpLs) 

223

In [88]:
arpLs

[['a85e-455a-5a7c', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['4cbd-8f79-1b53', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['e0d5-5efc-0288', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['00e0-4c52-032d', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['40b0-765f-6e47', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['04f9-f816-6b61', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['7486-e218-1afe', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['4cbd-8f79-1b76', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['fcaa-1470-f02e', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['4cbd-8f79-1b7a', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['b42e-999d-1f99', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['e062-90c9-1712', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['f079-596c-908a', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['7486-e218-0a99', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['0017-6112-279e', '1', 'Learned', 'GigabitEthernet1/0/23'],
 ['4419-b649-85b6', '1', 'Learned', 'GigabitEthernet1/0/24'],
 ['4cbd-

In [89]:
def mac2json(ls):
    """  带表头 col """
    rows = []
    for row in ls:
        #print(row)
        keys = ['mac','vlan','state', 'port']
        dc = dict(zip(keys, row))
        rows.append(dc)
    macJson = rows
    #print(macJson)
    return macJson 
   
mac2json(arpLs )

[{'mac': 'a85e-455a-5a7c',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '4cbd-8f79-1b53',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/23'},
 {'mac': 'e0d5-5efc-0288',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '00e0-4c52-032d',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '40b0-765f-6e47',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '04f9-f816-6b61',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/23'},
 {'mac': '7486-e218-1afe',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '4cbd-8f79-1b76',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/23'},
 {'mac': 'fcaa-1470-f02e',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/24'},
 {'mac': '4cbd-8f79-1b7a',
  'vlan': '1',
  'state': 'Learned',
  'port': 'GigabitEthernet1/0/23'},


In [90]:
len(mac2json(arpLs))

223

In [92]:
a=[{'mac': 'a85e-455a-5a7c', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '4cbd-8f79-1b53', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': 'e0d5-5efc-0288', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '00e0-4c52-032d', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '40b0-765f-6e47', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '04f9-f816-6b61', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': '4cbd-8f79-1b76', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': 'fcaa-1470-f02e', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '4cbd-8f79-1b7a', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': 'b42e-999d-1f99', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': 'e062-90c9-1712', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': 'f079-596c-908a', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '7486-e218-0a99', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '0017-6112-279e', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': '4419-b649-85b6', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '4cbd-8fd5-da85', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': 'e0d5-5ec3-191d', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': 'e062-9082-2258', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': '4cbd-8f2c-9597', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '4cbd-8f2c-9592', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/24'}, {'mac': '4cbd-8fd5-dab2', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}, {'mac': '4cbd-8f79-1b2a', 'vlan': '1', 'state': 'Learned', 'port': 'GigabitEthernet1/0/23'}]
len(a)

22

In [25]:


output = '''
                  display arp
IP ADDRESS      MAC ADDRESS  EXPIRE(M) TYPE INTERFACE      VPN-INSTANCE         VLAN 
------------------------------------------------------------------------------
192.168.11.1    80fb-06c9-6db9         I -  Vlanif11
192.168.11.199  b42e-9968-4090  11     D-0  Eth0/0/1  11
192.168.88.253  4cbd-8f90-7198  1      D-0  Eth0/0/11  88
192.168.88.161  Incomplete      0      D-0  Eth0/0/1  88
192.168.88.253  4cbd-8f90-7198  1      D-0  Eth0/0/11  88
192.168.88.161  Incomplete      0      D-0  Eth0/0/1  88
192.168.88.52   Incomplete      0      D-0  Eth0/0/1  88
192.168.0.1     80fb-06c9-6db9         I -  Vlanif100
192.168.0.45    2cea-7feb-b118  20     D-0  Eth0/0/20  100
192.168.0.252   f4ee-0823-956a  20     D-0  Eth0/0/24  100
192.168.0.177   9ca6-15d8-f7a2  20     D-0  Eth0/0/24  100
192.168.0.107   2cea-7fed-051c  18     D-0  Eth0/0/24  100
192.168.0.99    e0d5-5efa-1b79  20     D-0  Eth0/0/24  100
192.168.0.189   8ca6-dfc4-490d  20     D-0  Eth0/0/24  100
192.168.0.11    0017-6112-b286  20     D-0  Eth0/0/24  100
192.168.0.114   f4b5-2018-50e7  20     D-0  Eth0/0/24  100
10.0.0.2        80fb-06c9-6db9         I -  Vlanif200
10.0.0.1        0c82-6858-7222  20     D-0  GE0/0/3  200
192.168.201.1   80fb-06c9-6db9         I -  Vlanif201
------------------------------------------------------------------------------
Total:173       Dynamic:151     Static:0    Interface:22   
<3700>    
'''

f = open('display-arp.template')
template = TextFSM(f)
outTab = template.ParseText(tabStr)
print (outTab)

[['192.168.11.1', '80fb-06c9-6db9', 'I - ', 'Vlanif11', ''], ['192.168.11.199', 'b42e-9968-4090', 'D-0', 'Eth0/0/1', '  11'], ['192.168.11.30', '4419-b641-7795', 'D-0', 'Eth0/0/1', '  11'], ['192.168.11.189', '0800-2740-3822', 'D-0', 'Eth0/0/1', '  11'], ['192.168.12.1', '80fb-06c9-6db9', 'I - ', 'Vlanif12', ''], ['192.168.13.1', '80fb-06c9-6db9', 'I - ', 'Vlanif13', ''], ['192.168.14.1', '80fb-06c9-6db9', 'I - ', 'Vlanif14', ''], ['192.168.14.13', '8269-c0e2-cbe1', 'D-0', 'Eth0/0/11', '  14'], ['192.168.15.1', '80fb-06c9-6db9', 'I - ', 'Vlanif15', ''], ['192.168.15.254', '8ca6-df43-6b43', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.33', '7427-eae8-fef9', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.177', '4ced-fb6c-ef9f', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.200', '408d-5c8f-2aca', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.88', 'b42e-99e6-3c49', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.63', '4ced-fb41-fba5', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.231', 'b42e-999d-1f99', 'D-0', 'Eth0/0/5

In [26]:
len(outTab)

169

In [27]:
def hwArpRows(tabStr,tpl="display-arp.template"):
    f = open(tpl)
    template = TextFSM(f)
    outTab = template.ParseText(tabStr)
    return  outTab


In [28]:
print(tabStr)

display arp
IP ADDRESS      MAC ADDRESS  EXPIRE(M) TYPE INTERFACE      VPN-INSTANCE         VLAN 
------------------------------------------------------------------------------
192.168.11.1    80fb-06c9-6db9         I -  Vlanif11
192.168.11.199  b42e-9968-4090  9      D-0  Eth0/0/1  11
192.168.11.30   4419-b641-7795  9      D-0  Eth0/0/1  11
192.168.11.189  0800-2740-3822  9      D-0  Eth0/0/1  11
192.168.12.1    80fb-06c9-6db9         I -  Vlanif12
192.168.13.1    80fb-06c9-6db9         I -  Vlanif13
192.168.14.1    80fb-06c9-6db9         I -  Vlanif14
192.168.14.13   8269-c0e2-cbe1  4      D-0  Eth0/0/11  14
192.168.15.1    80fb-06c9-6db9         I -  Vlanif15
192.168.15.254  8ca6-df43-6b43  9      D-0  Eth0/0/5  15
192.168.15.33   7427-eae8-fef9  9      D-0  Eth0/0/5  15
192.168.15.177  4ced-fb6c-ef9f  9      D-0  Eth0/0/5  15
192.168.15.200  408d-5c8f-2aca  9      D-0  Eth0/0/5  15
192.168.15.88   b42e-99e6-3c49  17     D-0  Eth0/0/5  15
192.168.15.63   4ced-fb41-fba5  17     D-0  

In [88]:
def mac2tab(tn):
    """ 返回交换 arp表"""
    tabLs = []
    tn.write(b" display mac-address\n")
    while 1 :
        outLs = tn.expect([b"<.+?>",b"\r\r\n---- More ----"],timeout=5) #//[b"\r\r\n---- More ----"]
        if outLs[0] >= 0:
            #还有下一个页
            tn.write(b" ")
            tabLs.append(outLs[2].decode('ascii'))
            print(outLs)
        if outLs[0] < 0:
            # 完成读取
            print(outLs)
            tabLs.append(outLs[2].decode('ascii'))
            break
    return tabLs
tabLs = mac2tab(tn)
                      
tabLs

(1, <re.Match object; span=(1726, 1743), match=b'\r\r\n---- More ----'>, b' display mac-address\r\r\nMAC Address      VLAN ID    State            Port/NickName            Aging\r\r\nb83a-080e-f650   2          Learned          GE1/0/9                  Y\r\r\n04f9-f816-6b61   11         Learned          GE1/0/11                 Y\r\r\n3c6a-4852-c1fe   11         Learned          GE1/0/11                 Y\r\r\n4419-b641-7795   11         Learned          GE1/0/3                  Y\r\r\n5254-0033-e4b7   11         Learned          GE1/0/1                  Y\r\r\n5254-00c1-4efa   11         Learned          GE1/0/1                  Y\r\r\n5254-00d4-636c   11         Learned          GE1/0/1                  Y\r\r\n5869-6c01-dd73   11         Learned          GE1/0/1                  Y\r\r\n5cc9-99ea-55e0   11         Learned          GE1/0/11                 Y\r\r\n80fb-06c9-6db9   11         Learned          GE1/0/2                  Y\r\r\n9ca6-15d8-f7a2   11         Learned          GE1

(1, <re.Match object; span=(1717, 1734), match=b'\r\r\n---- More ----'>, b'\r\r               \r4cbd-8fd5-da70   34         Learned          GE1/0/11                 Y\r\r\n4cbd-8fd5-da85   34         Learned          GE1/0/11                 Y\r\r\n4cbd-8fd5-dab0   34         Learned          GE1/0/11                 Y\r\r\n4cbd-8fd5-dab2   34         Learned          GE1/0/11                 Y\r\r\n4cbd-8fd5-db1e   34         Learned          GE1/0/11                 Y\r\r\n4cbd-8fd5-dbd4   34         Learned          GE1/0/11                 Y\r\r\n5475-959c-548b   34         Learned          GE1/0/11                 Y\r\r\n5850-ed6d-2f60   34         Learned          GE1/0/11                 Y\r\r\n5850-ed6d-2f7b   34         Learned          GE1/0/11                 Y\r\r\n5869-6c01-dd73   34         Learned          GE1/0/11                 Y\r\r\n70b5-e874-4644   34         Learned          GE1/0/11                 Y\r\r\n7486-e218-0a99   34         Learned          GE1/0/11    

[' display mac-address\r\r\nMAC Address      VLAN ID    State            Port/NickName            Aging\r\r\nb83a-080e-f650   2          Learned          GE1/0/9                  Y\r\r\n04f9-f816-6b61   11         Learned          GE1/0/11                 Y\r\r\n3c6a-4852-c1fe   11         Learned          GE1/0/11                 Y\r\r\n4419-b641-7795   11         Learned          GE1/0/3                  Y\r\r\n5254-0033-e4b7   11         Learned          GE1/0/1                  Y\r\r\n5254-00c1-4efa   11         Learned          GE1/0/1                  Y\r\r\n5254-00d4-636c   11         Learned          GE1/0/1                  Y\r\r\n5869-6c01-dd73   11         Learned          GE1/0/1                  Y\r\r\n5cc9-99ea-55e0   11         Learned          GE1/0/11                 Y\r\r\n80fb-06c9-6db9   11         Learned          GE1/0/2                  Y\r\r\n9ca6-15d8-f7a2   11         Learned          GE1/0/1                  Y\r\r\nacb9-2f24-2384   11         Learned         

In [96]:
len(tabLs)

11

In [94]:
tabStr = ''.join(tabLs)
tabStr

' display mac-address\r\r\nMAC Address      VLAN ID    State            Port/NickName            Aging\r\r\nb83a-080e-f650   2          Learned          GE1/0/9                  Y\r\r\n04f9-f816-6b61   11         Learned          GE1/0/11                 Y\r\r\n3c6a-4852-c1fe   11         Learned          GE1/0/11                 Y\r\r\n4419-b641-7795   11         Learned          GE1/0/3                  Y\r\r\n5254-0033-e4b7   11         Learned          GE1/0/1                  Y\r\r\n5254-00c1-4efa   11         Learned          GE1/0/1                  Y\r\r\n5254-00d4-636c   11         Learned          GE1/0/1                  Y\r\r\n5869-6c01-dd73   11         Learned          GE1/0/1                  Y\r\r\n5cc9-99ea-55e0   11         Learned          GE1/0/11                 Y\r\r\n80fb-06c9-6db9   11         Learned          GE1/0/2                  Y\r\r\n9ca6-15d8-f7a2   11         Learned          GE1/0/1                  Y\r\r\nacb9-2f24-2384   11         Learned          

In [98]:
print( list2str(tabLs) ) 

 display mac-address
MAC Address      VLAN ID    State            Port/NickName            Aging
b83a-080e-f650   2          Learned          GE1/0/9                  Y
04f9-f816-6b61   11         Learned          GE1/0/11                 Y
3c6a-4852-c1fe   11         Learned          GE1/0/11                 Y
4419-b641-7795   11         Learned          GE1/0/3                  Y
5254-0033-e4b7   11         Learned          GE1/0/1                  Y
5254-00c1-4efa   11         Learned          GE1/0/1                  Y
5254-00d4-636c   11         Learned          GE1/0/1                  Y
5869-6c01-dd73   11         Learned          GE1/0/1                  Y
5cc9-99ea-55e0   11         Learned          GE1/0/11                 Y
80fb-06c9-6db9   11         Learned          GE1/0/2                  Y
9ca6-15d8-f7a2   11         Learned          GE1/0/1                  Y
acb9-2f24-2384   11         Learned          GE1/0/1                  Y
b42e-9968-4090   11         Learned    

In [234]:
                      
outs = tn.expect([b"\\x1b\[0m All\s+",b"\\n\\r\\n\\r\\r.*?\\r\\n\\r\\r\\n\\r.*?#"],timeout=5) 
outs

(-1, None, b' ')

In [92]:
def list2str(tabLs): 
    """ 多行合并成字符串"""
    tabStr = ''.join(tabLs)
    linp = '----\n\r\r'
    num = tabStr.find (linp)+len(linp)
    tabStr = tabStr[num:]
    tabStr =  re.sub("\\n\\r\\n\\r\\rTotal Entries:.*?\\r\\n\\r\\r\\n\\r.*?#", "", tabStr) #尾巴
    tabStr =  re.sub("\\x1b\[.*2K\\r\\r", "", tabStr) #翻页字符
    tabStr =  re.sub("\\r\\r", "", tabStr) #换行
    return tabStr
tabStr = list2str(tabLs)

In [93]:
print(tabStr)

ay mac-address
MAC Address      VLAN ID    State            Port/NickName            Aging
b83a-080e-f650   2          Learned          GE1/0/9                  Y
04f9-f816-6b61   11         Learned          GE1/0/11                 Y
3c6a-4852-c1fe   11         Learned          GE1/0/11                 Y
4419-b641-7795   11         Learned          GE1/0/3                  Y
5254-0033-e4b7   11         Learned          GE1/0/1                  Y
5254-00c1-4efa   11         Learned          GE1/0/1                  Y
5254-00d4-636c   11         Learned          GE1/0/1                  Y
5869-6c01-dd73   11         Learned          GE1/0/1                  Y
5cc9-99ea-55e0   11         Learned          GE1/0/11                 Y
80fb-06c9-6db9   11         Learned          GE1/0/2                  Y
9ca6-15d8-f7a2   11         Learned          GE1/0/1                  Y
acb9-2f24-2384   11         Learned          GE1/0/1                  Y
b42e-9968-4090   11         Learned          

In [379]:
z = ''.join(tabLs)
z[-19:]

's: 409\r\n\r\r\n\rRuijie#'

In [382]:
tabStr

'   1  0020.6BA3.4D7C     Dynamic     GigabitEthernet 0/1      \n   1  00D8.610C.3D58     Dynamic     GigabitEthernet 0/1      \n   1  00E0.0B48.A717     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4C37.ACE4     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4C3C.A0DB     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4C52.032D     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4C57.9160     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4C85.826D     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4CFF.2C14     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4D95.1326     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4F03.2E24     Dynamic     GigabitEthernet 0/1      \n   1  00E0.4F03.C582     Dynamic     GigabitEthernet 0/1      \n   1  00E0.7070.DE71     Dynamic     GigabitEthernet 0/1      \n   1  00E0.FC09.BCF9     Dynamic     GigabitEthernet 0/1      \n   1  0259.E77D.711A     Dynamic     GigabitEthernet 0/1      \n   1  0800.2740.3822     Dynamic     Gi

In [100]:

def mac2rows(tabStr,tpl="display-mac-s5560s.template"):
    f = open(tpl)
    template = TextFSM(f)
    outTab = template.ParseText(tabStr)
    return  outTab

In [101]:
tabStr = list2str(tabLs)
outTab = mac2rows(tabStr)
outTab

[['b83a-080e-f650', '2', 'Learned', 'GE1/0/9'],
 ['04f9-f816-6b61', '11', 'Learned', 'GE1/0/11'],
 ['3c6a-4852-c1fe', '11', 'Learned', 'GE1/0/11'],
 ['4419-b641-7795', '11', 'Learned', 'GE1/0/3'],
 ['5254-0033-e4b7', '11', 'Learned', 'GE1/0/1'],
 ['5254-00c1-4efa', '11', 'Learned', 'GE1/0/1'],
 ['5254-00d4-636c', '11', 'Learned', 'GE1/0/1'],
 ['5869-6c01-dd73', '11', 'Learned', 'GE1/0/1'],
 ['5cc9-99ea-55e0', '11', 'Learned', 'GE1/0/11'],
 ['80fb-06c9-6db9', '11', 'Learned', 'GE1/0/2'],
 ['9ca6-15d8-f7a2', '11', 'Learned', 'GE1/0/1'],
 ['acb9-2f24-2384', '11', 'Learned', 'GE1/0/1'],
 ['b42e-9968-4090', '11', 'Learned', 'GE1/0/3'],
 ['e0d5-5efa-1b79', '11', 'Learned', 'GE1/0/1'],
 ['020f-ed77-7960', '14', 'Learned', 'GE1/0/11'],
 ['80fb-06c9-6db9', '14', 'Learned', 'GE1/0/2'],
 ['4ced-fb6c-ef9f', '15', 'Learned', 'GE1/0/5'],
 ['7427-eae8-fef9', '15', 'Learned', 'GE1/0/5'],
 ['80fb-06c9-6db9', '15', 'Learned', 'GE1/0/2'],
 ['8ca6-df43-6b43', '15', 'Learned', 'GE1/0/5'],
 ['b42e-999d-1f99

In [85]:
        
def mac2json(ls):
    """  带表头 col """
    rows = []
    for row in ls:
        #print(row)
        keys = ['vlan','mac','type', 'port']
        #--转换 大小写
        
        dc = dict(zip(keys, row))
        dc['mac'] =dc['mac'].replace(".", "-").lower()
        rows.append(dc)
    macJson = rows#json.dumps(rows, sort_keys=True, indent=4, separators=(',', ': '))
    print(macJson)
    return macJson 
   
mac2json(outTab)

NameError: name 'outTab' is not defined

In [ ]:
mac2json()

In [387]:
print(tabStr)

   1  0020.6BA3.4D7C     Dynamic     GigabitEthernet 0/1      
   1  00D8.610C.3D58     Dynamic     GigabitEthernet 0/1      
   1  00E0.0B48.A717     Dynamic     GigabitEthernet 0/1      
   1  00E0.4C37.ACE4     Dynamic     GigabitEthernet 0/1      
   1  00E0.4C3C.A0DB     Dynamic     GigabitEthernet 0/1      
   1  00E0.4C52.032D     Dynamic     GigabitEthernet 0/1      
   1  00E0.4C57.9160     Dynamic     GigabitEthernet 0/1      
   1  00E0.4C85.826D     Dynamic     GigabitEthernet 0/1      
   1  00E0.4CFF.2C14     Dynamic     GigabitEthernet 0/1      
   1  00E0.4D95.1326     Dynamic     GigabitEthernet 0/1      
   1  00E0.4F03.2E24     Dynamic     GigabitEthernet 0/1      
   1  00E0.4F03.C582     Dynamic     GigabitEthernet 0/1      
   1  00E0.7070.DE71     Dynamic     GigabitEthernet 0/1      
   1  00E0.FC09.BCF9     Dynamic     GigabitEthernet 0/1      
   1  0259.E77D.711A     Dynamic     GigabitEthernet 0/1      
   1  0800.2740.3822     Dynamic     GigabitEthernet 0/

In [393]:


output = '''
12  00E0.4C57.9160     Dynamic     GigabitEthernet 0/17     
  12  00E0.4C85.826D     Dynamic     GigabitEthernet 0/23     
  12  00E0.4CFF.2C14     Dynamic     GigabitEthernet 0/22     
  12  00E0.4D95.1326     Dynamic     GigabitEthernet 0/19     
  12  00E0.4F03.2E24     Dynamic     GigabitEthernet 0/23     
  12  00E0.4F03.C582     Dynamic     GigabitEthernet 0/22     
  12  00E0.7070.DE71     Dynamic     GigabitEthernet 0/23     
  12  00E0.FC09.BCF9     Dynamic     GigabitEthernet 0/17     
  12  0259.E77D.711A     Dynamic     GigabitEthernet 0/17     
  12  0800.2740.3822     Dynamic     GigabitEthernet 0/17     
  12  081F.7150.30DE     Dynamic     GigabitEthernet 0/22     
  12  0840.F360.A0D8     Dynamic     GigabitEthernet 0/22        
'''

f = open('show-mac.template')
template = TextFSM(f)
outTab = template.ParseText(tabStr)
print (outTab)

[['1', '0020.6BA3.4D7C', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00D8.610C.3D58', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.0B48.A717', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4C37.ACE4', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4C3C.A0DB', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4C52.032D', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4C57.9160', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4C85.826D', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4CFF.2C14', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4D95.1326', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4F03.2E24', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.4F03.C582', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.7070.DE71', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '00E0.FC09.BCF9', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '0259.E77D.711A', 'Dynamic', 'GigabitEthernet 0/1'], ['1', '0800.2740.3822', 'Dynamic', 'GigabitEthernet 0/4'], ['1', '0C9D.920F.C668', 'Dynamic', 'GigabitEthernet 0/1

In [394]:
len(outTab)

408

### 任意读取

In [11]:
out = tn.read_very_eager().decode('ascii')#读取缓冲区内的数据不等待
out

''

### 关闭连接


In [221]:

tn.close()

In [44]:
tmp = json.dumps(outTab, sort_keys=True, indent=4, separators=(',', ': '))
print(tmp)

[
    [
        "00e0-4c37-ace4",
        "34",
        "Eth0/0/11"
    ],
    [
        "e0d5-5ef7-0e13",
        "20",
        "Eth0/0/9"
    ],
    [
        "00e0-4cff-2c14",
        "34",
        "Eth0/0/11"
    ],
    [
        "7427-eae8-fef9",
        "15",
        "Eth0/0/5"
    ],
    [
        "acb9-2f24-2384",
        "88",
        "Eth0/0/1"
    ],
    [
        "e0d5-5e56-7edb",
        "17",
        "Eth0/0/7"
    ],
    [
        "5cde-3414-a887",
        "26",
        "Eth0/0/13"
    ],
    [
        "f4b5-203b-2404",
        "26",
        "Eth0/0/16"
    ],
    [
        "70b5-e874-5844",
        "34",
        "Eth0/0/11"
    ],
    [
        "b42e-9904-6dda",
        "34",
        "Eth0/0/11"
    ],
    [
        "f4b5-2024-f44c",
        "34",
        "Eth0/0/11"
    ],
    [
        "f000-006c-8155",
        "34",
        "Eth0/0/11"
    ],
    [
        "f000-006c-8167",
        "34",
        "Eth0/0/11"
    ],
    [
        "b42e-99e6-3c49",
        "15",
       

In [43]:
int('100')

100

In [46]:
for row in outTab:


    keys = ['mac', 'vlan', 'port']

    dictionary = dict(zip(keys, row))
    print(dictionary)

{'mac': '00e0-4c37-ace4', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'e0d5-5ef7-0e13', 'vlan': '20', 'port': 'Eth0/0/9'}
{'mac': '00e0-4cff-2c14', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '7427-eae8-fef9', 'vlan': '15', 'port': 'Eth0/0/5'}
{'mac': 'acb9-2f24-2384', 'vlan': '88', 'port': 'Eth0/0/1'}
{'mac': 'e0d5-5e56-7edb', 'vlan': '17', 'port': 'Eth0/0/7'}
{'mac': '5cde-3414-a887', 'vlan': '26', 'port': 'Eth0/0/13'}
{'mac': 'f4b5-203b-2404', 'vlan': '26', 'port': 'Eth0/0/16'}
{'mac': '70b5-e874-5844', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'b42e-9904-6dda', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f4b5-2024-f44c', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f000-006c-8155', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f000-006c-8167', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'b42e-99e6-3c49', 'vlan': '15', 'port': 'Eth0/0/5'}
{'mac': 'e062-9087-a15a', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '4419-b649-83af', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '00e0-4f03-2e24', 'vl